In [87]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

In [71]:
# Load file into df
df = pd.read_csv('03_Library Systembook.csv')
df.head(5)


,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID
0,1.0,Catcher in the Rye,"""20/02/2023""",25/02/2023,2 weeks,1.0
1,2.0,Lord of the rings the two towers,"""24/03/2023""",21/03/2023,2 weeks,2.0
2,3.0,Lord of the rings the return of the kind,"""29/03/2023""",25/03/2023,2 weeks,3.0
3,4.0,The hobbit,"""02/04/2023""",25/03/2023,2 weeks,4.0
4,5.0,Dune,"""02/04/2023""",25/03/2023,2 weeks,5.0


In [72]:
# Covert IDs to integers
df['Id'] = df['Id'].astype('Int64')
df['Customer ID'] = df['Customer ID'].astype('Int64')

In [73]:
# Format dates columns
df['Book checkout'] = pd.to_datetime(df['Book checkout'].str.replace('"', ''), dayfirst = True, errors='coerce')
df['Book Returned'] = pd.to_datetime(df['Book Returned'], dayfirst = True, errors='coerce')

In [74]:
# Drop empty rows
df.dropna(how='all', inplace=True)

In [75]:
# Identify books where returns are before checkouts
df['Incorrect Date'] = df['Book Returned'] < df['Book checkout']

In [76]:
# Standardise book names
df['Books'] = df['Books'].str.strip().str.title()
df.head(10)

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID,Incorrect Date
0,1,Catcher In The Rye,2023-02-20,2023-02-25,2 weeks,1,False
1,2,Lord Of The Rings The Two Towers,2023-03-24,2023-03-21,2 weeks,2,True
2,3,Lord Of The Rings The Return Of The Kind,2023-03-29,2023-03-25,2 weeks,3,True
3,4,The Hobbit,2023-04-02,2023-03-25,2 weeks,4,True
4,5,Dune,2023-04-02,2023-03-25,2 weeks,5,True
5,6,Little Women,2023-04-02,2023-05-01,2 weeks,1,False
6,7,It,2063-04-10,2023-04-03,2 weeks,6,True
7,8,Misery,2023-04-15,2023-04-03,2 weeks,7,True
8,9,Catch 22,2023-04-15,2023-04-16,2 weeks,7,False
9,10,Animal Farm,2023-04-20,2023-04-24,2 weeks,2,False


In [77]:
# Identify unique book names
unique_books = df['Books'].unique()
print(unique_books)

['Catcher In The Rye' 'Lord Of The Rings The Two Towers'
 'Lord Of The Rings The Return Of The Kind' 'The Hobbit' 'Dune'
 'Little Women' 'It' 'Misery' 'Catch 22' 'Animal Farm' '1984'
 'East Of Eden' 'America Is In The Heart' 'Wuthering Heights' 'Dark Tales'
 'The Bloody Chamber' 'Les Miserables' 'Dracula' 'Frankenstein' nan]


In [78]:
# Fix title spelling
df['Books'] = df['Books'].str.replace('return of the kind', 'Return of the King', case=False)

In [ ]:
# Replace all unknown values with 0
df = df.fillna(0)

In [80]:
# Convert days allowed to borrow and create due column
df['Book checkout'] = pd.to_datetime(df['Book checkout'])
df['Days allowed to borrow'] = df['Days allowed to borrow'].str.extract('(\d+)').astype(int)*7
df['Due Date'] = df['Book checkout'] + pd.to_timedelta(df['Days allowed to borrow'], unit='D')

In [82]:
# Flag dates that are clearly in the future or too far in the past
df['Valid_Date'] = df['Book checkout'].between('2000-01-01', '2024-12-31')

In [84]:
# Filter to valid records
filtered_df = df.loc[(df['Incorrect Date'] == False) & (df['Valid_Date'] == True)]

In [85]:
# Standardize columns: "Book checkout" -> "book_checkout"
filtered_df.columns = filtered_df.columns.str.lower().str.replace(' ', '_')

In [86]:
filtered_df

,id,books,book_checkout,book_returned,days_allowed_to_borrow,customer_id,incorrect_date,due_date,valid_date
0,1,Catcher In The Rye,2023-02-20,2023-02-25,14,1,False,2023-03-06,True
5,6,Little Women,2023-04-02,2023-05-01,14,1,False,2023-04-16,True
8,9,Catch 22,2023-04-15,2023-04-16,14,7,False,2023-04-29,True
9,10,Animal Farm,2023-04-20,2023-04-24,14,2,False,2023-05-04,True
10,11,1984,2023-04-23,2023-04-27,14,8,False,2023-05-07,True
11,12,Little Women,2023-04-02,2023-05-01,14,1,False,2023-04-16,True
12,13,East Of Eden,2023-04-30,2023-05-05,14,2,False,2023-05-14,True
13,14,America Is In The Heart,2023-05-01,2023-05-07,14,3,False,2023-05-15,True
14,15,Wuthering Heights,2023-05-01,2023-05-10,14,9,False,2023-05-15,True
15,16,Dark Tales,2023-05-15,2023-06-01,14,2,False,2023-05-29,True


In [89]:
# Connection details
SERVER = 'STUDENT06'
DATABASE = 'LibraryProject'
DRIVER = 'ODBC Driver 17 for SQL Server'

# Connection string
connection_string = (
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"Trusted_Connection=yes;"
)

# Create the SQLAlchemy engine
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}")

# Test the connection
try:
    with engine.connect() as conn:
        print("Successfully connected to the server!")
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to the server!


In [90]:
# Move data to SQL Server
table_name = 'cleaned_library_data'
try:
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print("Success! The table '{table_name}' has been created in your SSMS database.")
except Exception as e:
    print(" Error uploading to SQL Server: {e}")

Success! The table '{table_name}' has been created in your SSMS database.
